In [4]:
import pandas as pd
import psycopg2
import time
import logging
from psycopg2.extras import execute_values
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from flask import Flask, request, send_file, jsonify
import os

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# File processing function
def processing_file(file_path, sheet_name=None):
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        logging.info("File loaded successfully. Here's a preview:")
        print(df.head())
        return df
    except FileNotFoundError:
        logging.error(f"The file {file_path} was not found.")
    except ValueError as e:
        logging.error(f"ValueError: {e}. Check if the file has accessible sheets and data.")
    except Exception as e:
        logging.error(f"An error occurred: {e}")

# Database connection function
def connect_to_db(retries=5, delay=1):
    for attempt in range(retries):
        try:
            conn = psycopg2.connect(
                dbname="postgres",
                user="nana",
                password="Twin@2017",
                host="localhost",
                port="5432"
            )
            conn.autocommit = True
            return conn
        except psycopg2.OperationalError as e:
            if attempt < retries - 1:
                logging.warning(f"Database connection failed; retrying in {delay} seconds... ({attempt+1}/5)")
                time.sleep(delay)
            else:
                logging.error("Could not connect to the database.")
                raise

# Data insertion function to PostgreSQL
def insert_data(events_data):
    required_columns = {'user_id', 'event_type', 'event_properties', 'timestamp'}
    if not required_columns.issubset(events_data.columns):
        logging.error("Data does not contain the required columns.")
        return

    try:
        conn = connect_to_db()
        cursor = conn.cursor()

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS events_data (
                user_id TEXT,
                event_type TEXT,
                event_properties TEXT,
                timestamp TIMESTAMP
            )
        ''')

        data_to_insert = [(row['user_id'], row['event_type'], row['event_properties'], row['timestamp'])
                          for _, row in events_data.iterrows()]

        insert_query = '''
            INSERT INTO events_data (user_id, event_type, event_properties, timestamp)
            VALUES %s
        '''
        execute_values(cursor, insert_query, data_to_insert)
        logging.info("Data inserted successfully.")
    except Exception as e:
        logging.error(f"Failed to insert data: {e}")
    finally:
        if cursor is not None:
            cursor.close()
        if conn is not None:
            conn.close()

# Function to export DataFrame to Excel
def export_results_to_excel(df, output_file):
    try:
        df.to_excel(output_file, index=False)
        logging.info(f'Results exported to {output_file}')
    except Exception as e:
        logging.error(f"Failed to export results: {e}")

# Machine learning pipeline
def train_and_evaluate(events_data):
    # Assuming 'churn' column exists or has been added to events_data
    if 'churn' in events_data.columns:
        label_encoder = LabelEncoder()
        events_data['event_type_encoded'] = label_encoder.fit_transform(events_data['event_type'])

        # Ensure 'timestamp' column is in datetime format
        events_data['timestamp'] = pd.to_datetime(events_data['timestamp'], errors='coerce')

        # Extracting features from timestamp
        events_data['hour'] = events_data['timestamp'].dt.hour
        events_data['day'] = events_data['timestamp'].dt.day
        events_data['month'] = events_data['timestamp'].dt.month

        # Features and target
        X = events_data[['event_type_encoded', 'hour', 'day', 'month']].copy()
        y = events_data['churn']

        # Train/test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Random Forest model
        clf = RandomForestClassifier(n_estimators=100)
        clf.fit(X_train, y_train)

        # Predict and evaluate
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        logging.info(f'Accuracy: {accuracy * 100:.2f}%')
    else:
        logging.warning("'churn' column is missing; please add it based on churn criteria.")

# Flask application for file upload and results export
app = Flask(__name__)

# Ensure the 'uploads' directory exists
UPLOAD_FOLDER = 'uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

@app.route('/upload', methods=['POST'])
def upload_file():
    file = request.files.get('file')
    if not file:
        return jsonify({"error": "No file provided"}), 400

    file_path = os.path.join(UPLOAD_FOLDER, file.filename)
    file.save(file_path)

    # Process the uploaded Excel file
    events_data = processing_file(file_path, sheet_name='events')
    if events_data is None:
        return jsonify({"error": "File processing failed"}), 500

    # Perform data processing and export results
    export_results_to_excel(events_data, 'results.xlsx')

    # Ensure results file exists before sending
    if not os.path.exists('results.xlsx'):
        return jsonify({"error": "Results file not found"}), 500

    return send_file('results.xlsx', as_attachment=True, mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')

if __name__ == '__main__':
    app.run(debug=True)

# Cohort analysis
def create_cohort_analysis(events_data):
    events_data['timestamp'] = pd.to_datetime(events_data['timestamp'], errors='coerce')

    # Create cohort DataFrame
    cohort_df = events_data.groupby('user_id').agg({
        'event_type': 'count', 
        'timestamp': 'min'
    }).reset_index()

    # Convert 'timestamp' to cohort by month
    cohort_df['cohort'] = cohort_df['timestamp'].dt.to_period('M')
    print(cohort_df)

# Example usage
if __name__ == '__main__':
    # Load and process the file
    file_path = '/Users/nana/Downloads/events_data.xlsx'
    events_data = processing_file(file_path, sheet_name='events')

    if events_data is not None:
        insert_data(events_data)
        train_and_evaluate(events_data)
        create_cohort_analysis(events_data)


 * Serving Flask app '__main__'
 * Debug mode: on


2024-11-13 15:41:49,109 - INFO - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
2024-11-13 15:41:49,109 - INFO - Press CTRL+C to quit
2024-11-13 15:41:49,147 - INFO -  * Restarting with watchdog (fsevents)
Traceback (most recent call last):
  File "/Users/nana/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 15, in <module>
    from ipykernel import kernelapp as app
  File "/Users/nana/anaconda3/lib/python3.11/site-packages/ipykernel/__init__.py", line 5, in <module>
    from .connect import *  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nana/anaconda3/lib/python3.11/site-packages/ipykernel/connect.py", line 11, in <module>
    import jupyter_client
  File "/Users/nana/anaconda3/lib/python3.11/site-packages/jupyter_client/__init__.py", line 8, in <module>
    from .asynchronous import AsyncKernelClient  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

SystemExit: 1

/Users/nana/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
